https://github.com/openai/whisper

In [12]:
import IPython.display as ipd
import librosa as rs
import time

In [2]:
import whisper

In [21]:
path = "./data/speech_male_kor.wav"

x,_ = rs.load(path,16000)

ipd.display(ipd.Markdown('## input'))
ipd.display(ipd.Audio(x, rate=16000))


/tmp/ipykernel_4075761/2610938252.py:3: FutureWarning: Pass sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  x,_ = rs.load(path,16000)


## input

'ㄴㅐㅇㅣㄹ'

In [23]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio(path)
audio = whisper.pad_or_trim(audio)

model = whisper.load_model("tiny")
model = model.to("cpu")

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)



# detect the spoken language
_, probs = model.detect_language(mel)
#print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
tic = time.time()
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

toc = time.time()
print("{} | {:.3f} sec".format("tiny cpu",toc-tic))

RuntimeError: "slow_conv2d_cpu" not implemented for 'Half'

In [20]:
# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio(path)
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

for size in ["tiny","base","small","medium","large"] : 

    model = whisper.load_model(size)

    # detect the spoken language
    _, probs = model.detect_language(mel)
    #print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    tic = time.time()
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)

    # print the recognized text
    print(result.text)

    toc = time.time()
    print("{} | {:.3f} sec".format(size,toc-tic))

내일 새벽 사회 전국으로 비부름이 확대 되겠습니다.
tiny | 0.111 sec


100%|███████████████████████████████████████| 139M/139M [00:17<00:00, 8.36MiB/s]


내일 새벽 4일 전국으로 비구름이 확대되겠습니다.
base | 0.231 sec


100%|███████████████████████████████████████| 461M/461M [00:57<00:00, 8.41MiB/s]


내일 새벽사에 전국으로 비부름이 확대되겠습니다.
small | 0.339 sec


100%|█████████████████████████████████████| 1.42G/1.42G [03:03<00:00, 8.35MiB/s]


내일 새벽 4일 전국으로 비구름이 확대되겠습니다.
medium | 0.602 sec


100%|█████████████████████████████████████| 2.87G/2.87G [06:10<00:00, 8.34MiB/s]


내일 새벽 사이 전국으로 비구름이 확대되겠습니다.
large | 0.805 sec


type|size|time(sec)|result
---|---|---|---
16kHz|입력 30초 단위|4.09|(정답)내일 새벽 사이 전국으로 비구름이 확대되겠습니다.
tiny|72MB|0.111|내일 새벽 사회 전국으로 비부름이 확대 되겠습니다.
base|139MB|0.231|내일 새벽 4일 전국으로 비구름이 확대되겠습니다.
small|461MB|0.339|내일 새벽사에 전국으로 비부름이 확대되겠습니다.
medium|1.42GB|0.062|내일 새벽 4일 전국으로 비구름이 확대되겠습니다.
large|2.87GB|0.805|내일 새벽 사이 전국으로 비구름이 확대되겠습니다.

In [26]:
model

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0): ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=384, out_features=384, bias=True)
          (key): Linear(in_features=384, out_features=384, bias=False)
          (value): Linear(in_features=384, out_features=384, bias=True)
          (out): Linear(in_features=384, out_features=384, bias=True)
        )
        (attn_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1536, out_features=384, bias=True)
        )
        (mlp_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      )
      (1): ResidualAttentionBlock(
        (attn): Mult